Data downloaded from (access data 09/21/2025): https://www.uniprot.org/help/embeddings

In [1]:
import os
import math
import h5py
import numpy as np
from typing import Callable, Any, Dict, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, random_split

import matplotlib.pyplot as plt

## Reproducibility & speed

In [2]:
def set_seed(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

## Training & eval helpers &  Data loading

In [3]:
def build_loaders(X: torch.Tensor, batch_size: int = 256, num_workers: int = 2):
    N = len(X)
    n_train = int(0.8 * N)
    n_val = int(0.1 * N)
    n_test = N - n_train - n_val
    ds = TensorDataset(X)
    train_set, val_set, test_set = random_split(
        ds, [n_train, n_val, n_test], generator=torch.Generator().manual_seed(42)
    )

    def make_loader(dset, shuffle=False):
        return DataLoader(
            dset,
            batch_size=batch_size,
            shuffle=shuffle,
            pin_memory=True,
            num_workers=num_workers,
            drop_last=False,
        )

    return make_loader(train_set, True), make_loader(val_set), make_loader(test_set)


def load_h5_embeddings(path: str) -> torch.Tensor:
    with h5py.File(path, "r") as f:
        protein_ids = list(f.keys())
        embeddings = [f[pid][()] for pid in protein_ids]
    X = torch.tensor(np.stack(embeddings), dtype=torch.float32)
    return X


## Losses

In [4]:
def normalized_mean_squared_error(
    reconstruction: torch.Tensor,
    original_input: torch.Tensor,
) -> torch.Tensor:
    return (
        ((reconstruction - original_input) ** 2).mean(dim=1) / (original_input**2).mean(dim=1)
    ).mean()


def normalized_L1_loss(
    latent_activations: torch.Tensor,
    original_input: torch.Tensor,
) -> torch.Tensor:
    return (latent_activations.abs().sum(dim=1) / original_input.norm(dim=1)).mean()


def autoencoder_loss(
    reconstruction: torch.Tensor,
    original_input: torch.Tensor,
    latent_activations: torch.Tensor,
    l1_weight: float,
) -> torch.Tensor:
    return (
        normalized_mean_squared_error(reconstruction, original_input)
        + normalized_L1_loss(latent_activations, original_input) * l1_weight
    )

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
AMP = True  # turn on mixed precision
GRAD_CLIP_NORM = 1.0  # set to None to disable
USE_COSINE = True  # cosine LR with warmup


# =========================
# Non-trainable LayerNorm
# =========================
def layer_norm_no_affine(
    x: torch.Tensor, eps: float = 1e-5
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Non-trainable LayerNorm over the last dim; returns (x_norm, mean, std).
    """
    mu = x.mean(dim=-1, keepdim=True)
    x_center = x - mu
    var = (x_center ** 2).mean(dim=-1, keepdim=True)
    std = torch.sqrt(var + eps)
    return x_center / std, mu, std


# =========================
# Weight tying wrapper
# =========================
class TiedTranspose(nn.Module):
    def __init__(self, linear: nn.Linear):
        super().__init__()
        self.linear = linear

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        assert self.linear.bias is None
        return F.linear(x, self.linear.weight.t(), None)

    @property
    def weight(self) -> torch.Tensor:
        return self.linear.weight.t()

    @property
    def bias(self) -> torch.Tensor:
        return self.linear.bias


# =========================
# Sparse activations
# =========================
class TopK(nn.Module):
    """
    Top-k sparse activation.

    Keeps the k largest values per row (last dimension) and sets all
    others to zero. Optionally applies a post-activation function
    (default: ReLU) to the selected values before reinserting them.
    """

    def __init__(self, k: int, postact_fn: Callable = nn.ReLU()) -> None:
        super().__init__()
        self.k = int(k)
        self.postact_fn = postact_fn

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor of shape [..., d].

        Returns:
            Tensor of the same shape as x, with only the top-k values
            (per last dimension) kept and transformed by postact_fn;
            all other entries are zero.
        """
        topk = torch.topk(x, k=self.k, dim=-1)
        values = self.postact_fn(topk.values)
        result = torch.zeros_like(x)
        result.scatter_(-1, topk.indices, values)
        return result


def _sample_gumbel(shape, device, eps=1e-20):
    """
    Draw i.i.d. samples from a standard Gumbel(0, 1) distribution.

    Args:
        shape: output tensor shape.
        device: torch device to place the samples on.
        eps: small constant for numerical stability.

    Returns:
        Tensor of Gumbel noise with the given shape.
    """
    u = torch.rand(shape, device=device)
    return -torch.log(-torch.log(u + eps) + eps)


class GumbelTopK(nn.Module):
    """Top-k with Gumbel; tau linearly anneals to tau_end then becomes hard Top-k."""
    def __init__(
        self,
        k: int,
        postact_fn: Callable = nn.ReLU(),
        tau_start: float = 1.0,
        tau_end: float = 1e-4,
        anneal_steps: int = 100,
        use_straight_through: bool = True,
    ) -> None:
        super().__init__()
        assert k >= 1 and tau_start > 0 and tau_end > 0 and anneal_steps >= 1
        self.k = int(k)
        self.postact_fn = postact_fn
        self.tau_start = float(tau_start)
        self.tau_end = float(tau_end)
        self.anneal_steps = int(anneal_steps)
        self.use_straight_through = bool(use_straight_through)
        self.register_buffer("step", torch.zeros((), dtype=torch.long))
        self.register_buffer("tau", torch.tensor(self.tau_start, dtype=torch.float))

    @torch.no_grad()
    def _update_tau(self):
        """ Linearly anneals `tau` from tau_start → tau_end over anneal_steps.
        
        Clamps at tau_end.
        """
        s = int(self.step.item())
        frac = min(s, self.anneal_steps) / self.anneal_steps
        new_tau = self.tau_start + (self.tau_end - self.tau_start) * frac
        self.tau.fill_(max(self.tau_end, float(new_tau)))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        d = x.size(-1)
        assert self.k <= d, f"k={self.k} cannot exceed last dimension {d}"
        values = self.postact_fn(x)

        if self.training:
            with torch.no_grad():
                self.step.add_(1)
                self._update_tau()

            # Once tau ~ tau_end, deterministic top-k on raw scores
            if self.tau.item() <= self.tau_end + 1e-12:
                topk = torch.topk(x, k=self.k, dim=-1)
                mask = torch.zeros_like(x).scatter(-1, topk.indices, 1.0)
                return values * mask

            # Stochastic Gumbel Top-k with STE
            g = _sample_gumbel(x.shape, device=x.device)
            logits = (x + g) / self.tau.clamp(min=1e-8)
            soft = F.softmax(logits, dim=-1)
            topk = torch.topk(logits, k=self.k, dim=-1)
            hard_mask = torch.zeros_like(x).scatter(-1, topk.indices, 1.0)
            mask = (hard_mask - soft).detach() + soft if self.use_straight_through else soft
            return values * mask
        else:
            topk = torch.topk(x, k=self.k, dim=-1)
            mask = torch.zeros_like(x).scatter(-1, topk.indices, 1.0)
            return values * mask


# =========================
# Sparse Autoencoder
# =========================
class SparseAutoEncoder(nn.Module):
    """
    latents = activation(encoder(x - pre_bias) + latent_bias)
    recons  = decoder(latents) + pre_bias
    """
    def __init__(
        self,
        n_latents: int,
        n_inputs: int,
        activation: Callable = nn.ReLU(),
        tied: bool = True,
        normalize: bool = False,
        init_row_norm_decoder: bool = True,  # row-normalize decoder at init (stabilizes early training)
    ) -> None:
        super().__init__()
        self.pre_bias = nn.Parameter(torch.zeros(n_inputs))

        # ---- Encoder (no bias); Kaiming/He-init like tricks ----
        self.encoder: nn.Linear = nn.Linear(n_inputs, n_latents, bias=False)
        nn.init.kaiming_uniform_(self.encoder.weight, a=math.sqrt(5))

        self.latent_bias = nn.Parameter(torch.zeros(n_latents))

        # ---- Decoder: tied or free ----
        if tied:
            self.decoder: nn.Module = TiedTranspose(self.encoder)
        else:
            self.decoder = nn.Linear(n_latents, n_inputs, bias=False)
            nn.init.kaiming_uniform_(self.decoder.weight, a=math.sqrt(5))

        # Optionally start decoder rows normalized (OpenAI/saefarer friendly)
        if isinstance(self.decoder, nn.Linear) and init_row_norm_decoder:
            with torch.no_grad():
                self.decoder.weight.data = F.normalize(self.decoder.weight.data, p=2, dim=1)
        elif isinstance(self.decoder, TiedTranspose) and init_row_norm_decoder:
            with torch.no_grad():
                W = self.encoder.weight.data
                self.encoder.weight.data = F.normalize(W, p=2, dim=0)  # normalize columns so decoder rows are unit

        self.activation = activation
        self.normalize = normalize

        # Runtime stats (optional; mirrors saefarer/OpenAI idea)
        self.register_buffer("stats_last_nonzero", torch.zeros(n_latents, dtype=torch.long))
        self.register_buffer("latents_activation_frequency", torch.ones(n_latents, dtype=torch.float))
        self.register_buffer("latents_mean_square", torch.zeros(n_latents, dtype=torch.float))

    def encode_pre_act(self, x: torch.Tensor, latent_slice: slice = slice(None)) -> torch.Tensor:
        x = x - self.pre_bias
        latents_pre_act = F.linear(
            x, self.encoder.weight[latent_slice], self.latent_bias[latent_slice]
        )
        return latents_pre_act

    def preprocess(self, x: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, Any]]:
        if not self.normalize:
            return x, {}
        x_norm, mu, std = layer_norm_no_affine(x)
        return x_norm, {"mu": mu, "std": std}

    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, Any]]:
        x, info = self.preprocess(x)
        return self.activation(self.encode_pre_act(x)), info

    def decode(self, latents: torch.Tensor, info: Dict[str, Any] | None = None) -> torch.Tensor:
        ret = self.decoder(latents) + self.pre_bias
        if self.normalize:
            assert info is not None
            ret = ret * info["std"] + info["mu"]
        return ret

    def forward(self, x: torch.Tensor):
        x_proc, info = self.preprocess(x)
        latents_pre_act = self.encode_pre_act(x_proc)
        latents = self.activation(latents_pre_act)
        recons = self.decode(latents, info)

        # Update idle stats like OpenAI/saefarer
        self.stats_last_nonzero *= (latents == 0).all(dim=0).long()
        self.stats_last_nonzero += 1

        return latents_pre_act, latents, recons

## Train, and eval 

In [6]:
@torch.no_grad()
def evaluate_epoch(model, loader, l1_weight: float) -> Dict[str, float]:
    model.eval()
    losses, mses = [], []
    for (x,) in loader:
        x = x.to(DEVICE, non_blocking=True)
        latents_pre, latents, recons = model(x)
        loss = autoencoder_loss(recons, x, latents, l1_weight)
        mse = normalized_mean_squared_error(recons, x)
        losses.append(loss.item())
        mses.append(mse.item())
    return {"loss": float(np.mean(losses)), "nmse": float(np.mean(mses))}


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: DataLoader,
    *,
    epochs: int = 20,
    lr: float = 1e-3,
    l1_weight: float = 0.0,
    warmup_steps: int = 0,
):
    model.to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    scaler = torch.cuda.amp.GradScaler(enabled=AMP)

    if USE_COSINE:
        total_steps = epochs * max(1, len(train_loader))
        def lr_lambda(step):
            if step < warmup_steps:
                return max(1e-6, (step + 1) / max(1, warmup_steps))
            prog = (step - warmup_steps) / max(1, total_steps - warmup_steps)
            return 0.5 * (1 + math.cos(math.pi * min(1.0, max(0.0, prog))))
        sched = torch.optim.lr_scheduler.LambdaLR(opt, lr_lambda=lr_lambda)
    else:
        sched = None

    history = {
        "train_loss": [], "train_nmse": [],
        "val_loss": [], "val_nmse": [],
        "test_loss": [], "test_nmse": [],
    }

    global_step = 0
    for epoch in range(1, epochs + 1):
        model.train()
        train_losses, train_mses = [], []
        for (x,) in train_loader:
            x = x.to(DEVICE, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=AMP):
                latents_pre, latents, recons = model(x)
                loss_total = autoencoder_loss(recons, x, latents, l1_weight)
                nmse = normalized_mean_squared_error(recons, x)

            scaler.scale(loss_total).backward()

            if GRAD_CLIP_NORM is not None:
                scaler.unscale_(opt)
                torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP_NORM)

            scaler.step(opt)
            scaler.update()
            if sched is not None:
                sched.step()
            global_step += 1

            train_losses.append(loss_total.item())
            train_mses.append(nmse.item())

        # Epoch-end eval
        train_loss = float(np.mean(train_losses))
        train_nmse = float(np.mean(train_mses))
        val_metrics = evaluate_epoch(model, val_loader, l1_weight)
        test_metrics = evaluate_epoch(model, test_loader, l1_weight)

        history["train_loss"].append(train_loss)
        history["train_nmse"].append(train_nmse)
        history["val_loss"].append(val_metrics["loss"])
        history["val_nmse"].append(val_metrics["nmse"])
        history["test_loss"].append(test_metrics["loss"])
        history["test_nmse"].append(test_metrics["nmse"])

        print(
            f"Epoch {epoch:02d} | "
            f"train: loss {train_loss:.4f}, nmse {train_nmse:.4f} | "
            f"val: loss {val_metrics['loss']:.4f}, nmse {val_metrics['nmse']:.4f} | "
            f"test: loss {test_metrics['loss']:.4f}, nmse {test_metrics['nmse']:.4f}"
        )

    return history

In [ ]:
# =========================
# Plotting
# =========================
def plot_summary(histA, histB, histC, title="SAE variants comparison"):
    """
    Builds a 2x3 grid: rows = [Loss, NMSE], cols = [Train, Val, Test]
    Bars = [ReLU+L1, TopK, GumbelTopK] using the final-epoch values.
    """
    splits = ["train", "val", "test"]
    models = ["ReLU+L1", "TopK=64", "GumbelTopK=64"]

    # Final epoch values
    final = {
        "ReLU+L1": {
            "loss": [histA["train_loss"][-1], histA["val_loss"][-1], histA["test_loss"][-1]],
            "nmse": [histA["train_nmse"][-1], histA["val_nmse"][-1], histA["test_nmse"][-1]],
        },
        "TopK=64": {
            "loss": [histB["train_loss"][-1], histB["val_loss"][-1], histB["test_loss"][-1]],
            "nmse": [histB["train_nmse"][-1], histB["val_nmse"][-1], histB["test_nmse"][-1]],
        },
        "GumbelTopK=64": {
            "loss": [histC["train_loss"][-1], histC["val_loss"][-1], histC["test_loss"][-1]],
            "nmse": [histC["train_nmse"][-1], histC["val_nmse"][-1], histC["test_nmse"][-1]],
        },
    }

    fig, axes = plt.subplots(2, 3, figsize=(14, 6), constrained_layout=True)
    width = 0.25
    x = np.arange(len(models))

    for col, split in enumerate(splits):
        # Loss row
        ax = axes[0, col]
        vals = [final[m]["loss"][col] for m in models]
        ax.bar(x, vals, width)
        ax.set_title(f"{split.capitalize()} Loss")
        ax.set_xticks(x, models, rotation=20)
        ax.grid(True, axis="y", linestyle="--", alpha=0.3)

        # NMSE row
        ax = axes[1, col]
        vals = [final[m]["nmse"][col] for m in models]
        ax.bar(x, vals, width)
        ax.set_title(f"{split.capitalize()} NMSE")
        ax.set_xticks(x, models, rotation=20)
        ax.grid(True, axis="y", linestyle="--", alpha=0.3)

    fig.suptitle(title)
    plt.show()


# =========================
# Main
# =========================
def main():
    # ---- Load data ----
    file_path = "./data/per-protein.h5"
    X = load_h5_embeddings(file_path)  # [N, D]
    n_inputs = X.shape[1]

    # ---- Dataloaders ----
    train_loader, val_loader, test_loader = build_loaders(X, batch_size=256, num_workers=2)

    # ---- Common config ----
    epochs = 25
    lr = 1e-3
    warmup_steps = 200

    # ---- Model A: ReLU SAE (MSE + L1), no TopK ----
    model_a = SparseAutoEncoder(
        n_latents=256,
        n_inputs=n_inputs,
        activation=nn.ReLU(),
        tied=True,               # initialize decoder as encoderᵀ
        normalize=False,         # no LN in this variant
        init_row_norm_decoder=True,
    )
    histA = train_model(
        model_a, train_loader, val_loader, test_loader,
        epochs=epochs, lr=lr, l1_weight=1e-3, warmup_steps=warmup_steps
    )

    # ---- Model B: TopK=64 (MSE only) ----
    model_b = SparseAutoEncoder(
        n_latents=256,
        n_inputs=n_inputs,
        activation=TopK(k=64),
        tied=True,
        normalize=True,          # good practice with sparse activations
        init_row_norm_decoder=True,
    )
    histB = train_model(
        model_b, train_loader, val_loader, test_loader,
        epochs=epochs, lr=lr, l1_weight=0.0, warmup_steps=warmup_steps
    )

    # ---- Model C: GumbelTopK=64 (MSE only) ----
    model_c = SparseAutoEncoder(
        n_latents=256,
        n_inputs=n_inputs,
        activation=GumbelTopK(k=64, tau_start=1.0, tau_end=1e-8, anneal_steps=10_000),
        tied=True,
        normalize=True,
        init_row_norm_decoder=True,
    )
    histC = train_model(
        model_c, train_loader, val_loader, test_loader,
        epochs=epochs, lr=lr, l1_weight=0.0, warmup_steps=warmup_steps
    )

    # ---- Figure: compare final metrics ----
    plot_summary(histA, histB, histC, title="Sparse Autoencoder Variants (Loss & NMSE)")

if __name__ == "__main__":
    main()

/var/folders/zs/3r1nbv8s4qx7qvtcgk18pwhw0000gn/T/ipykernel_8559/362259851.py:28: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=AMP)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/var/folders/zs/3r1nbv8s4qx7qvtcgk18pwhw0000gn/T/ipykernel_8559/362259851.py:54: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=AMP):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch 01 | train: loss 1.4647, nmse 1.4607 | val: loss 0.8309, nmse 0.8286 | test: loss 0.8236, nmse 0.8214
Epoch 02 | train: loss 0.6381, nmse 0.6351 | val: loss 0.5290, nmse 0.5255 | test: loss 0.5179, nmse 0.5144
Epoch 03 | train: loss 0.4331, nmse 0.4290 | val: loss 0.3561, nmse 0.3513 | test: loss 0.3474, nmse 0.3426
Epoch 04 | train: loss 0.2907, nmse 0.2850 | val: loss 0.2439, nmse 0.2371 | test: loss 0.2368, nmse 0.2300
Epoch 05 | train: loss 0.2094, nmse 0.2015 | val: loss 0.1898, nmse 0.1810 | test: loss 0.1836, nmse 0.1747
Epoch 06 | train: loss 0.1698, nmse 0.1601 | val: loss 0.1614, nmse 0.1510 | test: loss 0.1557, nmse 0.1452
Epoch 07 | train: loss 0.1478, nmse 0.1368 | val: loss 0.1444, nmse 0.1330 | test: loss 0.1390, nmse 0.1275
Epoch 08 | train: loss 0.1343, nmse 0.1224 | val: loss 0.1330, nmse 0.1209 | test: loss 0.1278, nmse 0.1156
Epoch 09 | train: loss 0.1247, nmse 0.1122 | val: loss 0.1253, nmse 0.1126 | test: loss 0.1204, nmse 0.1077
Epoch 10 | train: loss 0.117